# 4304 Project Dashboard

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import ipywidgets as widgets
from IPython.display import display, clear_output

import pyfonts
from mpl_flags import Flags
from matplotlib.patches import Circle
from matplotlib.offsetbox import (TextArea, DrawingArea, AnnotationBbox, VPacker)

___________________

In [2]:
df = pd.read_csv('satcat.tsv', sep='\t', low_memory=False)

In [3]:
orgs = pd.read_csv("orgs.tsv", sep="\t", usecols=["#Code", "Class", "ShortName"]).drop(0)
orgs.rename(columns={"#Code": "Owner"}, inplace=True)

In [4]:
class_mapping = {
    "A": "Academic/Non-Profit",
    "B": "Business/Commercial",
    "C": "Civil Government",
    "D": "Defense/Military"
}

orbit_mapping = {
    'LEO': ['LEO/I', 'LLEO/I', 'LEO/P', 'LLEO/P', 'LEO/S', 'LEO/E', 'LEO/R', 'LLEO/R', 'LLEO/S', 'LLEO/E'],
    'MEO': ['MEO'],
    'GEO': ['GEO/NS', 'GEO/T', 'GEO/S', 'GEO/D', 'GEO/I', 'GEO/ID'],
    'GTO': ['GTO'],
    'HEO': ['HEO', 'VHEO', 'HEO/M'],
    'Deep Space': ['DSO', 'SO', 'CLO', 'TA'],
    'Unclassified': ['-']
}

def get_class_for_owner(owner):
    owners = owner.split("/")
    classes = orgs[orgs["Owner"].isin(owners)]["Class"].unique()
    
    return classes[0] if len(classes) > 0 else None
    
def map_orbit_type(oporbit):
    for category, types in orbit_mapping.items():
        if oporbit in types:
            return category
    return 'Other'

df['OrbitType'] = df['OpOrbit'].apply(map_orbit_type)
df["Class"] = df["Owner"].apply(get_class_for_owner)

In [5]:
df = df.merge(orgs[['Owner', 'ShortName']], on='Owner', how='left')

In [6]:
df['State'] = df['State'].replace("SU", "RU")

In [7]:
df['ShortName'] = df['ShortName'].replace("SpaceX/Seattle", "SpaceX")
df['ShortName'] = df['ShortName'].replace("SpaceX Tourists", "SpaceX")
df['ShortName'] = df['ShortName'].replace("SpaceX/McGregor", "SpaceX")

In [8]:
font = pyfonts.load_font('https://github.com/google/fonts/blob/main/ofl/spacemono/SpaceMono-Regular.ttf?raw=true')
spacemono_bold = pyfonts.load_font('https://github.com/google/fonts/blob/main/ofl/spacemono/SpaceMono-Bold.ttf?raw=true')
lato = pyfonts.load_font('https://github.com/google/fonts/blob/main/ofl/lato/Lato-SemiBold.ttf?raw=true')
num_bold = pyfonts.load_font('https://github.com/google/fonts/blob/main/ofl/economica/Economica-Bold.ttf?raw=true')
chakra = pyfonts.load_font('https://github.com/google/fonts/blob/main/ofl/chakrapetch/ChakraPetch-Regular.ttf?raw=true')
chakra_bold = pyfonts.load_font('https://github.com/google/fonts/blob/main/ofl/chakrapetch/ChakraPetch-Bold.ttf?raw=true')
quan_bold = pyfonts.load_font('https://github.com/google/fonts/blob/main/ofl/quantico/Quantico-Bold.ttf?raw=true')

__________________________

## CurvedText class taken from StackOverflow (https://stackoverflow.com/a/44521963)

## Visualization 1

In [11]:
def get_circle_properties(count, scale_factor = 0.0055):
    radius = np.sqrt(count)* scale_factor
    return radius

def add_stars(ax, num_stars=100):
    np.random.seed(42)
    star_x = np.random.uniform(-15, 15, num_stars)
    star_y = np.random.uniform(-15, 15, num_stars)
    star_sizes = np.random.uniform(2, 8, num_stars)
    star_alpha = np.random.uniform(0.2, 0.5, num_stars)
    ax.scatter(star_x, star_y, s=star_sizes, c='white', alpha=star_alpha, marker='*', zorder = 0)

def get_color(country):
    cmap = plt.cm.tab20
    if country == 'Other':
        return '#9ca3a3' 
    
    # Create unique numeric value for each country using hash
    seed = abs(hash(country)) % (2**32) 
    np.random.seed(seed)  
    return cmap(np.random.rand())

labels = {
    "US": "U.S.A.",
    "CN": "China",
    "RU": "Russia",
    "Other": "Other",
    "UK": "U.K.",
    "F": "France",
    "J": "Japan",
    'D': 'Germany',
    'KR': 'South Korea',
    'E': 'Spain',
    'IL': 'Israel',
    'IN': 'India',
    'L': 'Luxembourg',
    'CA': 'Canada',
    'SG': 'Singapore',
    'I': 'Italy',
    'TW': 'Taiwan'
}

In [27]:
class_selector = widgets.Dropdown(
    options=[(v, k) for k, v in class_mapping.items()] + [("All Classes", "All")],
    value='All',
    description='Class:'
)

active_checkbox = widgets.Checkbox(value=True, description='Active')
alltime_checkbox = widgets.Checkbox(value=False, description='All time')

# Link checkboxes for mutual exclusion
def update_checkboxes(change):
    # Prevent both from being deselected
    if not active_checkbox.value and not alltime_checkbox.value:
        # If both are unchecked, force the last changed one back to True
        change['owner'].value = True
        return
    
    # Maintain mutual exclusion when checking
    if change['new']:  # Only act on positive changes
        if change['owner'] == active_checkbox:
            alltime_checkbox.value = False
        else:
            active_checkbox.value = False

active_checkbox.observe(update_checkboxes, names='value')
alltime_checkbox.observe(update_checkboxes, names='value')

# Data processing function
def preprocess_data(selected_class, active_status):
    # Filter by class
    if selected_class == 'All':
        class_filter = df['Class'].notna()
    else:
        class_filter = df['Class'] == selected_class
    
    # Filter by status
    if active_status:
        status_filter = df['Status'] == 'O'
    else:
        status_filter = df['Status'].notna()  # Include all statuses

    state_filter = ~df['State'].str.startswith('I-', na=False)
    
    # Combine all filters
    filtered = df[
        class_filter & 
        status_filter & 
        df['Type'].str.contains('P', na=False) & 
        state_filter
    ]
    return filtered

# Plotting function
def plot_data(payloads_filtered):
    payload_counts = payloads_filtered['State'].value_counts()
    how_many = 8  # 3x3 grid (8 countries + Other)
    top_countries = payload_counts.head(how_many).to_dict()
    other_count = payload_counts.iloc[how_many:].sum()

    # Sort countries descending and add Other last
    sorted_countries = sorted(top_countries.items(), key=lambda x: x[1], reverse=True)
    countries = sorted_countries + [("Other", other_count)]
    
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.set_frame_on(False)
    ax.set_xticks([])
    ax.set_yticks([])
    fig.patch.set_facecolor('#030c36')
    ax.set_facecolor('#030c36')

    # Calculate bubble properties
    max_count = max([c[1] for c in countries])
    scale_factor = 0.045
    radii = [get_circle_properties(c[1], scale_factor) for c in countries]
    max_radius = max(radii)
    
    # Grid layout parameters
    grid_cols = 3
    grid_rows = 3
    horizontal_spacing = 10
    vertical_spacing = 10
    start_x = -10
    start_y = -10
    
    # Calculate grid positions
    grid_positions = [
        (start_x + col*horizontal_spacing, 
         start_y + row*vertical_spacing)
        for row in range(grid_rows-1, -1, -1)
        for col in range(grid_cols)
    ]

    # Track bounds for dynamic scaling
    min_x, max_x = float('inf'), -float('inf')
    min_y, max_y = float('inf'), -float('inf')

    flags = Flags("circle")

    for (country, count), (grid_x, grid_y) in zip(countries, grid_positions):
        radius = get_circle_properties(count, scale_factor)
        x, y = grid_x, grid_y
        
        # Update bounds tracking
        effective_radius = radius + 1.5
        min_x = min(min_x, x - effective_radius)
        max_x = max(max_x, x + effective_radius)
        min_y = min(min_y, y - effective_radius)
        max_y = max(max_y, y + effective_radius)
        
        # Draw bubble
        color = get_color(country)
        bubble = plt.Circle((x, y), radius, color=color, alpha=0.9, zorder=1)
        if country == "Other":
            ax.add_patch(bubble)

        # Add flag (skip for 'Other')
        if country != "Other":
            # Map country codes to ISO standards
            if country == "UK":
                country_code = "GB"
            elif country == "F":
                country_code = "FR"
            elif country == "J":
                country_code = "JP"
            elif country == "D":
                country_code = "DE"  # Germany
            elif country == "E":
                country_code = "ES"  # Spain
            elif country == "I":
                country_code = "IT"  # Italy
            elif country == "L":
                country_code = "LU"  # Luxembourg
            else:
                country_code = country  # Use direct code for others

            if radius < 0.35:
                magic_rad = 55
            else: magic_rad = 30
            # Add flag overlay
            da = flags.get_drawing_area(country_code, wmax=radius*magic_rad)
            ab = AnnotationBbox(da, (x, y), frameon=False, 
                               box_alignment=(0.5, 0.5), zorder=2)
            ax.add_artist(ab)
        
        # Add labels
        ax.text(x, y - radius - 0.8, 
                labels.get(country, country), 
                ha='center', va='top', color='white',
                fontsize=10, fontweight='bold', zorder=3)
        
        ax.text(x, y + radius + 0.5, 
                f"{count}", 
                ha='center', va='bottom', 
                color='white', fontsize=10, 
                fontweight='bold', zorder=3)

    # Set dynamic bounds with padding
    padding = 2
    ax.set_xlim(min_x - padding, max_x + padding)
    ax.set_ylim(min_y - padding, max_y + padding)
    ax.set_aspect('equal', adjustable='datalim')
    
    plt.show()


# Interactive update handler
output = widgets.Output()

def update_plots(change):
    with output:
        clear_output(wait=True)
        filtered_data = preprocess_data(class_selector.value, active_checkbox.value)
        plot_data(filtered_data)

# Initial display
display(widgets.VBox([
    class_selector,
    widgets.HBox([active_checkbox, alltime_checkbox]),
    output
]))

# Trigger initial plot
update_plots(None)

# Set up observers for dynamic updates
class_selector.observe(update_plots, names='value')
active_checkbox.observe(update_plots, names='value')
alltime_checkbox.observe(update_plots, names='value')